In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

## Get Indeks Link

In [4]:
def getIndeksLink(links, page, cat_link, category, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "https://indeks.kompas.com/"+cat_link+"/"+date+"/"+str(page)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.findAll('div', class_="article__list clearfix")
    for post in contentDiv:
        link = [post.find('a', href=True)['href'], category]
        links.append(link)
        
    el_page =  soup.find('div', class_="paging__wrap clearfix")
    if el_page:
        a_page = el_page.findAll('div', class_='paging__item')[-1].find('a')
        if  el_page.findAll('div', class_='paging__item')[-1].find('a', class_="paging__link paging__link--active"):
            max_page = page
        else:
            max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
            
        if page < max_page:
            links = getIndeksLink(links, page+1, cat_link, category, date)
        
    return links

## Get Detail Berita

In [24]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #articleid
        articles['id'] = int(soup.find("meta", attrs={'name':'cXenseParse:articleid'})['content'])

        #category
        articles['category'] = link[1]
        articles['url'] = url

        #extract subcategory from breadcrumb
        bc = soup.find('ul', class_="breadcrumb__wrap")
        articles['subcategory'] = bc.findAll('li')[2].text

        #article
        article = soup.find("div", class_="read__content")

        #extract date
        pubdate = soup.find("meta", attrs={"name":"content_PublishedDate"})['content']
        pubdate = pubdate.strip(' \t\n\r')
        articles['pubdate']=datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%d %H:%M:%S"), "%Y-%m-%d %H:%M:%S")

        #extract author
        articles['author'] = soup.find('div', class_="read__author").text

        #extract title
        articles['title'] = soup.find('h1', class_="read__title").text      

        #source
        articles['source'] = 'kompas'

        #extract comments count
        #articles['comments'] = 0
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')  # Last I checked this was necessary.
        options.add_argument('--disable-extensions')

        driver = webdriver.Chrome("../chromedriver.exe", chrome_options=options)
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html5lib')

        comment = soup.find('div', class_="span4 comments-count tright")
        comment_num = comment.text.replace('Ada ', '')
        comment_num = comment_num.replace(' komentar untuk artikel ini', '')
        articles['comments'] = comment_num

        #extract tags
        tags = soup.find("meta", attrs={'name':'content_tag'})['content']
        articles['tags'] = tags

        #extract images
        articles['images'] = soup.find('meta', attrs={'property':'og:image'})['content']

        #hapus all script
        for script in article.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [12]:
links = getIndeksLink([], 1, 'news', 'news', '2018-07-01')

page  1
https://indeks.kompas.com/news/2018-07-01/1
page  2
https://indeks.kompas.com/news/2018-07-01/2
page  3
https://indeks.kompas.com/news/2018-07-01/3
page  4
https://indeks.kompas.com/news/2018-07-01/4
page  5
https://indeks.kompas.com/news/2018-07-01/5
page  6
https://indeks.kompas.com/news/2018-07-01/6


In [25]:
details = getDetailBerita(links[0:2])
details

[{'author': 'Kontributor Kompas TV Babel, Rahmatul Fauza',
  'category': 'news',
  'comments': '3',
  'content': 'PANGKALPINANG, KOMPAS.com — Menteri Pemuda dan Olahraga Imam Nahrawi meminta proses hukum terkait kasus pemukulan pada anaknya tetap diteruskan."Saya berharap kepolisian akan terus melanjutkan proses hukumnya," kata Imam seusai menyaksikan MXGP 2018 seri Pangkalpinang, Kepulauan Bangka Belitung, Minggu (1/7/2018).Imam menyayangkan kekerasan yang dialami anaknya saat menonton pertandingan sepak bola antara Persija melawan Persebaya beberapa waktu lalu.Menurut dia, seharusnya suporter tidak bertindak anarkistis. Jalur hukum ditempuh untuk memberikan pembelajaran kepada para suporter.Baca juga: Jakmania yang Diduga Pukul Anak Menpora Jadi Tersangka tetapi Tak Ditahan"Karena ini juga merupakan pembelajaran bagi suporter sepakbola yang seharusnya berlaku baik," ujar Imam.Imam mengatakan, walaupun akan ada upaya damai ataupun permohonan maaf dari Jakmania, proses hukum harus teta

In [26]:
df = pd.DataFrame(details)

In [27]:
df

,author,category,comments,content,id,images,pubdate,source,subcategory,tags,title,url
0,"Kontributor Kompas TV Babel, Rahmatul Fauza",news,3,"PANGKALPINANG, KOMPAS.com — Menteri Pemuda dan...",1618444,https://asset.kompas.com/crop/0x0:780x390/780x...,2018-07-01 23:06:22,kompas,Regional,,Menpora Minta Kasus Penganiayaan Anaknya Tetap...,http://regional.kompas.com/read/2018/07/01/230...
1,,news,1,"KOMPAS.com - Mahasiswa Indonesia di Mesir, Muh...",1618434,https://asset.kompas.com/crop/0x0:1000x667/780...,2018-07-01 22:26:36,kompas,Internasional,,"Jatuh dari Balkon Apartemen di Mesir, Mahasisw...",http://internasional.kompas.com/read/2018/07/0...


In [28]:
soup.findAll('a', class_="inner-link")

[<a class="inner-link" href="http://megapolitan.kompas.com/read/2018/07/01/11355491/jakmania-yang-diduga-pukul-anak-menpora-jadi-tersangka-tetapi-tak-ditahan" target="_self">Jakmania yang Diduga Pukul Anak Menpora Jadi Tersangka tetapi Tak Ditahan</a>,
 <a class="inner-link" href="http://megapolitan.kompas.com/read/2018/06/30/21255791/polisi-amankan-jakmania-yang-diduga-pukul-anak-menpora" target="_self">Polisi Amankan Jakmania yang Diduga Pukul Anak Menpora</a>,
 <a class="inner-link" href="http://kompas.com" target="_blank">Kompas.com</a>]